In [1]:
from collections import defaultdict
from openai import OpenAI
from collections import defaultdict, deque
from typing import Dict, Any, List, Tuple, Optional
import datetime
import os
import uuid
import datetime
import string
import random
from dataclasses import dataclass

# Python decorator that automatically creates helpful methods in NPC Factory
@dataclass
class NPCConfig:
    world_description: str
    character_description: str
    history_size: int = 10
    has_scratchpad: bool = False

class NPCFactoryError(Exception):
    """Base exception class for NPC Factory errors."""
    pass

class NPCNotFoundError(NPCFactoryError):
    """Raised when trying to interact with a non-existent NPC."""
    def __init__(self, npc_id: str):
        self.npc_id = npc_id
        super().__init__(f"NPC with ID '{npc_id}' not found")
        
@dataclass
class Message:
    sender: str
    content: str
    timestamp: datetime.datetime

class ChatArenaError(Exception):
    """Base exception class for Chat Arena errors."""
    pass

class ConversationNotFoundError(ChatArenaError):
    """Raised when trying to interact with a non-existent conversation."""
    def __init__(self, conversation_id: str):
        self.conversation_id = conversation_id
        super().__init__(f"Conversation with ID '{conversation_id}' not found")

class NPCNotFoundInArenaError(ChatArenaError):
    """Raised when trying to interact with a non-existent NPC in the arena."""
    def __init__(self, npc_id: str):
        self.npc_id = npc_id
        super().__init__(f"NPC with ID '{npc_id}' not found in arena")



In [2]:
class ChatArena:
    def __init__(self, npc_factory):
        """Initialize ChatArena with an NPCFactory instance."""
        self.npc_factory = npc_factory
        self.conversations = defaultdict(lambda: {
            'npc1_id': None,
            'npc2_id': None,
            'messages': [],
            'current_turn': None,
            'metrics': defaultdict(float)
        })
        self.metrics = defaultdict(lambda: defaultdict(float))

        # Map NPC IDs to their corresponding user IDs
        self.npc_user_ids: Dict[str, str] = {}
        
    def register_npc_in_arena(self, npc_id: str) -> str:
        """Register an NPC as a user in the arena.

        Args:
            npc_id: ID of the NPC to register

        Returns:
            user_id: The user ID assigned to this NPC

        Raises:
            NPCNotFoundInArenaError: If the NPC ID doesn't exist in factory
        """
        # <YOUR CODE HERE>
        '''
        Chong Ming 1: Because the NPCFactory is built for user-to-NPC interaction, each NPC needs a user_id so they can speak speak in the chat.
        This method creates a user account for the NPC (if not already done), returns the user_id, and saves it for reuse.
        '''
        # If already registered, return existing user_id
        if npc_id in self.npc_user_ids:
            return self.npc_user_ids[npc_id]
        
        # Otherwise, generate a unique username for the NPC (e.g. "npc_abcd1234")
        username = f"npc_{npc_id[:8]}"
        user_id = self.npc_factory.register_user(username)
        self.npc_user_ids[npc_id] = user_id
        return user_id
        # <MY CODE END HERE>
    
    def start_conversation(self, npc1_id: str, npc2_id: str,
                         initial_prompt: str = "Hello! Who are you?") -> str:
        """Start a conversation between two NPCs.

        Args:
            npc1_id: ID of the first NPC
            npc2_id: ID of the second NPC
            initial_prompt: Optional initial message to start the conversation

        Returns:
            conversation_id: Unique identifier for the conversation

        Raises:
            NPCNotFoundInArenaError: If either NPC ID doesn't exist
        """
        # Register both NPCs if they haven't been registered yet
        # <YOUR CODE HERE>
        '''
        Chong Ming 2: For the subsequent codes below <YOUR CODE HERE>, these are the objectives and reasons why write the codes this way
        Ensures both NPCs are registered as users.
        Sets up all IDs, initializes the conversation history, and notes whose turn is next.
        Adds the opening line (from NPC1 to NPC2).
        '''
        npc1_user_id = self.register_npc_in_arena(npc1_id)
        npc2_user_id = self.register_npc_in_arena(npc2_id)
        # <MY CODE END HERE>
        
        # Initialize conversation
        # <YOUR CODE HERE>

        # Create a unique conversation ID
        conversation_id = uuid.uuid4().hex

        # Set up the conversation record
        self.conversations[conversation_id] = {
            'npc1_id': npc1_id,
            'npc2_id': npc2_id,
            'npc1_user_id': npc1_user_id,
            'npc2_user_id': npc2_user_id,
            'messages': [],
            'current_turn': 'npc2',  # npc2 replies first (since npc1 initiates)
            'metrics': defaultdict(float)
        }
        # <MY CODE END HERE>
        
        # Add initial message if provided
        # <YOUR CODE HERE>
        self.conversations[conversation_id]['messages'].append(
            Message(
                sender=npc1_id,
                content=initial_prompt,
                timestamp=datetime.datetime.now()
            )
        )
        return conversation_id
        # <MY CODE END HERE>
    
    def run_turn(self, conversation_id: str) -> Tuple[bool, str]:
        """Run a single turn in the conversation.

        Args:
            conversation_id: ID of the conversation to progress

        Returns:
            Tuple of (success: bool, response: str)

        Raises:
            ConversationNotFoundError: If the conversation ID doesn't exist
        """
        # <YOUR CODE HERE>
        """Run a single turn in the conversation."""
        '''
        Chong Ming 3: Alternates which NPC speaks each turn to keep conversation going.
        Retrieves the last message, sends it to the next NPC (using user_id), and records the reply.
        Switches the turn for the following round.
        '''
        if conversation_id not in self.conversations:
            raise ConversationNotFoundError(conversation_id)

        convo = self.conversations[conversation_id]
        current_turn = convo['current_turn']

        # Determine who is talking (as user) and who is replying (as NPC)
        if current_turn == 'npc2':
            user_id = convo['npc2_user_id']
            npc_id = convo['npc1_id']
            last_message = convo['messages'][-1].content
            sender = convo['npc2_id']
            next_turn = 'npc1'
        else:  # current_turn == 'npc1'
            user_id = convo['npc1_user_id']
            npc_id = convo['npc2_id']
            last_message = convo['messages'][-1].content
            sender = convo['npc1_id']
            next_turn = 'npc2'

        # The current NPC (as user) sends the last message to the other NPC (as the "assistant")
        response = self.npc_factory.chat_with_npc(
            npc_id=npc_id,
            user_id=user_id,
            message=last_message
        )

        # Record the reply in the conversation history
        msg_obj = Message(
            sender=sender,
            content=response,
            timestamp=datetime.datetime.now()
        )
        convo['messages'].append(msg_obj)

        # Switch turns for next time
        convo['current_turn'] = next_turn

        return True, response
        # <MY CODE END HERE>
    
    def run_conversation(self, conversation_id: str, max_turns: int = 10,
                        verbose: bool = False) -> List[Message]:
        """Run a conversation for specified number of turns."""
        # <YOUR CODE HERE>
        '''
        Automates the conversation for a set number of turns. Set at 10 rounds.
        Calls run_turn repeatedly, optionally prints each reply, and returns the full message list.
        '''
        messages = []
        for i in range(max_turns):
            success, reply = self.run_turn(conversation_id)
            convo = self.conversations[conversation_id]
            messages.append(convo['messages'][-1])
            if verbose:
                print(f"Turn {i+1}: {reply}\n")
        return self.conversations[conversation_id]['messages']
    
    def evaluate_conversation(self, conversation_id: str) -> Dict[str, float]:
        """Evaluate a conversation, store and return metrics."""
        if conversation_id not in self.conversations:
            raise ConversationNotFoundError(conversation_id)

        # <YOUR CODE HERE>
        '''
        Computes basic stats for the conversation (e.g., average length).
        Stores metrics for future use.
        '''
        messages = self.conversations[conversation_id]['messages']
        # Example: Average answer length in words
        if messages:
            avg_length = sum(len(msg.content.split()) for msg in messages) / len(messages)
        else:
            avg_length = 0.0

        metrics = {'average_answer_length': avg_length}
        self.conversations[conversation_id]['metrics'] = metrics
        return metrics

    def get_conversation_history(self, conversation_id: str) -> List[Message]:
        """Get the full history of a conversation."""
        if conversation_id not in self.conversations:
            raise ConversationNotFoundError(conversation_id)

        return self.conversations[conversation_id]['messages']

In [3]:
class SimpleChatNPC:
    def __init__(self, client: OpenAI, model: str, config: NPCConfig):
        self.client = client
        self.model = model
        self.config = config
        self.chat_histories = defaultdict(lambda: deque(maxlen=config.history_size))

    def get_system_message(self) -> Dict[str, str]:
        """Returns the system message that defines the NPC's behavior."""
        character_description = self.config.character_description

        if self.config.has_scratchpad:
            character_description += """
You can use scratchpad for thinking before you answer: whatever you output in #SCRATCHPAD and #ANSWER won't be shown to anyone.
You start your output with #SCRATCHPAD and after you've done thinking, you #ANSWER"""

        return {
            "role": "system",
            "content": f"""WORLD SETTING: {self.config.world_description}
###
{character_description}"""
        }

    def chat(self, user_message: str, user_id: str) -> str:
        """Process a user message and return the NPC's response."""
        messages = [self.get_system_message()]

        # Add conversation history
        # Grab the previous conversation (history) between this user and the NPC.
        # If there is chat history, add it to the message list (so the LLM remembers what was said earlier).
        
        history = list(self.chat_histories[user_id])
        if history:
            messages.extend(history)

        # Add new user message
        # Make a dictionary for the new message (who said it and what they said).
        # Add it to the conversation history for this user.
        # Add it to the messages list to send to the LLM.
        
        user_message_dict = {
            "role": "user",
            "content": user_message
        }
        self.chat_histories[user_id].append(user_message_dict)
        messages.append(user_message_dict)

        # Ask LLM to generate a reply, using the chosen model, the conversation so far, and a set temperature.
        # Get the generated response (the NPC’s full reply, which may include a scratchpad depending if boolean is true or false).
        try:
            completion = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                temperature=0.6
            )

            response = completion.choices[0].message.content

            # Handle scratchpad if enabled
            # If scratchpads are enabled:
            # Look for the #SCRATCHPAD and #ANSWER markers in the response.
            # Extract only the part after #ANSWER (the actual answer for the user), and store it in response_clean variable.
            
            response_clean = response
            if self.config.has_scratchpad:
                import re
                scratchpad_match = re.search(r"#SCRATCHPAD(:?)(.*?)#ANSWER(:?)", response, re.DOTALL)
                if scratchpad_match:
                    response_clean = response[scratchpad_match.end():].strip()


            # Store response in history, including the scratchpad
            # Save the entire response (including scratchpad, if there is one) in the history for this user.
            
            self.chat_histories[user_id].append({
                "role": "assistant",
                "content": response
            })

            # Return the message to the user without a scratchpad
            return response_clean

        except Exception as e:
            return f"Error: {str(e)}"

In [4]:
class NPCFactory:

    # When create an NPCFactory, have to give:
    # client: the connection to the LLM service.
    # model: the name of the language model to use.
    # It sets up two dictionaries:
        # self.npcs: for storing all the NPCs you make (key: NPC ID, value: NPC object).
        # self.user_ids: for storing all users (key: username, value: user’s unique ID).
    def __init__(self, client: OpenAI, model: str):
        self.client = client
        self.model = model
        self.npcs: Dict[str, SimpleChatNPC] = {}
        self.user_ids: Dict[str, str] = {}  # username -> user_id mapping

    # Makes a random 8-letter string (e.g., “aBcDeFgH”) to use as a unique ID for users or NPCs.
    def generate_id(self) -> str:
        """Generate a random unique identifier."""
        return ''.join(random.choice(string.ascii_letters) for _ in range(8))

    # Alows add/register a new user by name.
    # If the name is already taken, it keeps adding a number at the end until the name is unique (like "Alice", "Alice_1", "Alice_2", etc.).
    # Creates a new user ID (random string), stores it, and returns it.
    def register_user(self, username: str) -> str:
        """Register a new user and return their unique ID.
        If username already exists, appends a numerical suffix."""
        base_username = username
        suffix = 1

        # Keep trying with incremented suffixes until we find an unused name
        while username in self.user_ids:
            username = f"{base_username}_{suffix}"
            suffix += 1

        user_id = self.generate_id()
        self.user_ids[username] = user_id
        return user_id

    #Create a new NPC with the world description, character description, memory size, and scratchpad option.
    # Generates a unique ID for this NPC.
    def register_npc(self, world_description: str, character_description: str,
                     history_size: int = 10, has_scratchpad: bool = False) -> str:
        """Create and register a new NPC, returning its unique ID."""
        npc_id = self.generate_id()
        
        # Creates a configuration object (NPCConfig) to neatly store all the NPC’s settings.
        config = NPCConfig(
            world_description=world_description,
            character_description=character_description,
            history_size=history_size,
            has_scratchpad=has_scratchpad
        )
        
        # Creates the actual NPC object (a chatbot) using all the details above.
        # Stores it in the npcs dictionary using its unique ID.
        # Returns the NPC’s ID so you can use it for chatting.
        
        self.npcs[npc_id] = SimpleChatNPC(self.client, self.model, config)
        return npc_id

    # Handles chatting! Code give it the NPC ID, user ID, and message.
    # If the NPC exists, it sends the message to that NPC, and returns the NPC’s reply.
    # If not, it throws an error.
    
    def chat_with_npc(self, npc_id: str, user_id: str, message: str) -> str:
        """Send a message to a specific NPC from a specific user.

        Args:
            npc_id: The unique identifier of the NPC
            user_id: The unique identifier of the user
            message: The message to send

        Returns:
            The NPC's response

        Raises:
            NPCNotFoundError: If the specified NPC doesn't exist
        """
        if npc_id not in self.npcs:
            raise NPCNotFoundError(npc_id)

        npc = self.npcs[npc_id]
        return npc.chat(message, user_id)

    # Go fetch the full chat history (list of all messages) between a specific user and NPC.
    # Returns the list (in order) so can see the whole conversation.
    
    def get_npc_chat_history(self, npc_id: str, user_id: str) -> list:
        """Retrieve chat history between a specific user and NPC.

        Args:
            npc_id: The unique identifier of the NPC
            user_id: The unique identifier of the user

        Returns:
            List of message dictionaries containing the chat history

        Raises:
            NPCNotFoundError: If the specified NPC doesn't exist
        """
        if npc_id not in self.npcs:
            raise NPCNotFoundError(npc_id)

        return list(self.npcs[npc_id].chat_histories[user_id])

In [5]:
with open("nebius_api_key", "r") as file:
    nebius_api_key = file.read().strip()

os.environ["NEBIUS_API_KEY"] = nebius_api_key



# Initialize the system
client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)

model = "meta-llama/Meta-Llama-3.1-405B-Instruct"

npc_factory = NPCFactory(client=client, model=model)
npc_arena = ChatArena(npc_factory)

# Create two NPCs
knight_id = npc_factory.register_npc(
    world_description="Medieval London, XIII century",
    character_description="A proud knight at Edward I's court",
    has_scratchpad=True
)

merchant_id = npc_factory.register_npc(
    world_description="Medieval London, XIII century",
    character_description="A wealthy merchant from the Hanseatic League",
    has_scratchpad=True
)

# Start a conversation between them
conv_id = npc_arena.start_conversation(knight_id, merchant_id)

# Run the conversation
messages = npc_arena.run_conversation(conv_id, max_turns=6, verbose=True)

Turn 1: Greetings, good fellow! I am Sir Edward de Montfort, a loyal knight in the service of our illustrious King Edward I. It is an honor to make your acquaintance. May I inquire as to your presence at court?

Turn 2: Sir Edward de Montfort, 'tis an honor to meet you, sir. I am Hermann von Stade, a humble merchant from the Hanseatic League. I have come to court seeking an audience with His Majesty, King Edward I, to discuss a proposal for increased trade between our league and the kingdom. We have a bounty of fine goods, including wool, furs, and amber, which I believe would be of great interest to the king and his people. I hope to negotiate a mutually beneficial agreement that would strengthen the bonds of commerce between our nations.

Turn 3: Hermann von Stade, a pleasure to make your acquaintance. Your proposal sounds intriguing, and I am certain that His Majesty would be interested in hearing more about the goods you have to offer. As you may know, the kingdom is always seeking